# Gallery Example: Er/M/1 Reentrant Queue

This example demonstrates an Erlang/M/1 queueing system with reentrant routing:
- **Arrivals**: Erlang distribution
- **Service**: Exponential service times
- **Servers**: 1 server
- **Scheduling**: FCFS
- **Reentrant**: Jobs switch class and pass through the queue again

In reentrant queues, jobs complete service and re-enter the queue as a different class.

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_erlm1_reentrant():
    """Create Er/M/1-Reentrant queueing model"""
    model = Network('Er/M/1-Reentrant')
    
    # Block 1: nodes
    source = Source(model, 'Source')
    queue = Queue(model, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model, 'Sink')
    
    # Block 2: classes
    oclass1 = OpenClass(model, 'Class1')
    oclass2 = OpenClass(model, 'Class2')
    
    # Erlang arrival for Class1, Class2 is disabled at source
    source.set_arrival(oclass1, Erlang.fit_mean_and_order(1, 5))
    source.set_arrival(oclass2, Disabled())
    
    # Different service rates for each class
    queue.set_service(oclass1, Exp(2))
    queue.set_service(oclass2, Exp(3))
    
    # Block 3: topology with class switching (reentrant)
    P = model.init_routing_matrix()
    P.add_route(oclass1, source, queue, 1.0)        # Arrivals go to queue as Class1
    P.add_class_switch(oclass1, oclass2, queue, queue, 1.0)  # Class1 switches to Class2 (reentrant)
    P.add_route(oclass2, queue, sink, 1.0)          # Class2 exits to sink
    model.link(P)
    
    return model

# Create the model
model = gallery_erlm1_reentrant()

## About Reentrant Queues

In reentrant queueing systems:
- Jobs pass through the same server multiple times
- Each pass may have different service characteristics (class switching)
- Models manufacturing systems with rework or multi-stage processing

In this model:
1. Jobs arrive as Class1 with Erlang(1, 5) inter-arrival times
2. After service (Exp(2)), they become Class2
3. Class2 jobs receive service (Exp(3)) and then exit

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)